# CommonRoad Scenario Designer Tutorial
This tutorial shows how to use the APIs for map conversion of the CommonRoad Scenario Designer.

## 1. Preparation
Before you proceed any further, make sure you have successfully installed the CommonRoad Scenario
Designer.

## 2. OSM to CommonRoad


In [4]:
import crdesigner.osm2cr.converter_modules.converter as converter
import crdesigner.osm2cr.converter_modules.cr_operations.export as ex
from crdesigner.osm2cr import config
from crdesigner.osm2cr.converter_modules.osm_operations.downloader import download_around_map

download_around_map(config.BENCHMARK_ID + '_downloaded.osm', 48.140289, 11.566272)

# open the map and convert it to a scenario
scenario = converter.GraphScenario(config.SAVE_PATH + config.BENCHMARK_ID + ".osm")

# draw and show the scenario
scenario.plot()

# save the scenario as commonroad file
scenario.save_as_cr(config.SAVE_PATH + config.BENCHMARK_ID + ".xml")
# save the scenario as a binary
scenario.save_to_file(config.SAVE_PATH + config.BENCHMARK_ID + ".pickle")

# view the generated
ex.view_xml(config.SAVE_PATH + config.BENCHMARK_ID + ".xml")

downloading map
writing custom bounds
reading File and creating graph
extract main layer


FileNotFoundError: [Errno 2] No such file or directory: 'files/ZAM_Test-1_1_T-1.osm'

## 3. OpenDRIVE to CommonRoad

## 3. Lanelet/Lanelet2 to CommonRoad and CommonRoad to Lanelet

## 3. CommonRoad to SUMO and Vice Versa